# **Notebook 01: Data Collection**
-----

### **Objective**

We collect raw data from multiple authoritative sources to build a comprehensive view of the UK tech startup ecosystem. This notebook establishes our data ingestion pipelines for company registrations, funding announcements, hiring signals, and product launches.

### **Data Sources**

| Source | Data Type | Update Frequency | API/Scraping |
|--------|-----------|------------------|--------------|
| Companies House | Incorporations, financials, directors | Daily | Free API |
| Tech.eu | Funding announcements | Daily | RSS/Scraping |
| Sifted | Funding news, analysis | Daily | RSS/Scraping |
| Product Hunt | Product launches | Daily | Free API |
| Job Boards | Job postings, headcount | Weekly | Scraping |

### **Technical Stack**

- **requests**: HTTP requests to APIs and web pages
- **pandas**: Data manipulation and storage
- **BeautifulSoup**: HTML parsing for web scraping
- **python-dotenv**: Environment variable management for API keys
- **SQLite**: Local database for raw data storage



---

In [1]:
# Import required libraries
import requests
import pandas as pd
import sqlite3
from datetime import datetime
import time
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()


False

### **Companies House API Integration**

- We begin by connecting to the Companies House API to pull UK company incorporation data. 

- Companies House is the UK's official registrar of companies, providing authoritative data on all registered businesses.

- We'll filter for technology companies incorporated between 2022-2025 to focus on early-stage startups.

- We have imported our core libraries and set up access to the Companies House API. 

- The API key is stored in a `.env` file (which we'll create next) to keep credentials secure and out of version control.

In [2]:
# Load environment variables
load_dotenv()

# Set up Companies House API credentials
API_KEY = os.getenv('COMPANIES_HOUSE_API_KEY', '')  # We'll add this to .env file
BASE_URL = 'https://api.company-information.service.gov.uk'

- We test our API connection by fetching data for a known company (Revolut). 

- The Companies House API uses HTTP Basic Authentication where the API key is the username and the password is left empty. 

- If successful, we'll see the company name and incorporation date.

In [3]:
# Test the API connection
def test_companies_house_connection():
    """
    Test our connection to Companies House API by fetching a known company
    """
    # Test with a well-known company (e.g., Revolut's UK entity)
    test_company_number = "08804411"  # Revolut Ltd
    
    url = f"{BASE_URL}/company/{test_company_number}"
    
    response = requests.get(url, auth=(API_KEY, ''))
    
    if response.status_code == 200:
        company_data = response.json()
        print("✓ Connection successful!")
        print(f"Company name: {company_data['company_name']}")
        print(f"Status: {company_data['company_status']}")
        print(f"Incorporated: {company_data['date_of_creation']}")
        return True
    else:
        print(f"✗ Connection failed. Status code: {response.status_code}")
        print(f"Error: {response.text}")
        return False

# Run the test
test_companies_house_connection()

✓ Connection successful!
Company name: REVOLUT LTD
Status: active
Incorporated: 2013-12-06


True

- We define Standard Industrial Classification (SIC) codes that identify technology companies. 

- These are the official UK government codes used to classify business activities. 

- We focus on software development, IT services, and data processing: the "core" tech startup sectors. 

- We also incorporate FinTech, AI/Data, Biotech, E-Commerce, etc. 

- Companies House requires all UK companies to declare their SIC codes at incorporation.

In [4]:
# Define SIC codes for technology companies
TECH_SIC_CODES = [
    # Core Software & IT
    '62011',  # Ready-made interactive leisure and entertainment software development
    '62012',  # Business and domestic software development
    '62020',  # Information technology consultancy activities
    '62090',  # Other information technology service activities
    '63110',  # Data processing, hosting and related activities
    '63120',  # Web portals
    '63990',  # Other information service activities (SaaS platforms)
    '58210',  # Publishing of computer games
    '58290',  # Other software publishing
    '26200',  # Manufacture of computers and peripheral equipment
    '95110',  # Repair of computers and peripheral equipment
    
    # FinTech
    '64190',  # Other monetary intermediation (digital banking, payment platforms)
    '64999',  # Financial intermediation not elsewhere classified (crypto, lending tech)
    '66190',  # Other activities auxiliary to financial intermediation (FinTech infrastructure)
    
    # HealthTech & BioTech
    '86900',  # Other human health activities (digital health, telemedicine)
    '72110',  # Research and experimental development on biotechnology
    
    # AI, Data & Analytics
    '70220',  # Business and other management consultancy (includes AI/ML consulting)
    '73110',  # Advertising agencies (MarTech, AdTech)
    '73120',  # Media representation (digital media platforms)
    
    # E-commerce & Marketplaces
    '82990',  # Other business support service activities (marketplace platforms)
    '47910',  # Retail sale via mail order or internet
]

print(f"Tracking {len(TECH_SIC_CODES)} tech-related SIC codes")
print("\nBreakdown by category:")
print(f"  Core Software/IT: 11 codes")
print(f"  FinTech: 3 codes")
print(f"  HealthTech: 2 codes")
print(f"  AI/Data: 3 codes")
print(f"  E-commerce: 2 codes")

Tracking 21 tech-related SIC codes

Breakdown by category:
  Core Software/IT: 11 codes
  FinTech: 3 codes
  HealthTech: 2 codes
  AI/Data: 3 codes
  E-commerce: 2 codes


- We build a search function that queries Companies House for companies incorporated within a specific date range and matching our tech SIC codes. 

- We search one SIC code at a time to avoid hitting API limits, and add a small delay between requests to be respectful of **rate limits (600 requests per 5 minutes)**.

- **Note**: We're testing with just January 2024 and 3 SIC codes first to verify the approach works before running the full search.

In [5]:
def search_companies_by_date(incorporated_from, incorporated_to, sic_codes):
    """
    Search for companies incorporated within a date range with specific SIC codes.
    Companies House search API has limitations, so we'll use advanced search.
    """
    
    companies = []
    
    # Companies House advanced search endpoint
    search_url = f"{BASE_URL}/advanced-search/companies"
    
    for sic_code in sic_codes:
        print(f"Searching SIC code: {sic_code}...", end=" ")
        
        params = {
            'incorporated_from': incorporated_from,
            'incorporated_to': incorporated_to,
            'sic_codes': sic_code,
            'size': 100  # Max results per page
        }
        
        try:
            response = requests.get(
                search_url, 
                auth=(API_KEY, ''),
                params=params
            )
            
            if response.status_code == 200:
                data = response.json()
                items = data.get('items', [])
                companies.extend(items)
                print(f"Found {len(items)} companies")
                
            else:
                print(f"Error: {response.status_code}")
            
            # Rate limiting: 600 requests per 5 minutes
            time.sleep(0.5)  # Be respectful to the API
            
        except Exception as e:
            print(f"Exception: {str(e)}")
            continue
    
    print(f"\nTotal companies found: {len(companies)}")
    return companies

# Test with a small date range first (January 2024)
test_companies = search_companies_by_date('2024-01-01', '2024-01-31', TECH_SIC_CODES[:3])

Searching SIC code: 62011... Found 100 companies
Searching SIC code: 62012... Found 100 companies
Searching SIC code: 62020... Found 100 companies

Total companies found: 300


- We will not keep an artificial page limit and fetch ALL results for each SIC code. 

- The function now includes a safety check for rate limiting (HTTP 429 status): if we hit the limit, we pause for 60 seconds before continuing. 

- This ensures we capture every company without missing any due to pagination caps.

In [7]:
def search_all_companies(incorporated_from, incorporated_to, sic_codes):
    """
    Search ALL companies without page limits - fetch until no more results.
    We'll monitor API responses and add safety checks for rate limits.
    """
    
    all_companies = []
    total_requests = 0
    
    for sic_code in sic_codes:
        print(f"\nSearching SIC code: {sic_code}")
        page = 0
        
        while True:  # Keep fetching until no more results
            search_url = f"{BASE_URL}/advanced-search/companies"
            
            params = {
                'incorporated_from': incorporated_from,
                'incorporated_to': incorporated_to,
                'sic_codes': sic_code,
                'size': 100,
                'start_index': page * 100
            }
            
            try:
                response = requests.get(
                    search_url,
                    auth=(API_KEY, ''),
                    params=params,
                    timeout=10
                )
                
                total_requests += 1
                
                if response.status_code == 200:
                    data = response.json()
                    items = data.get('items', [])
                    
                    if not items:  # No more results
                        print(f"  Completed: {page * 100} total companies")
                        break
                    
                    all_companies.extend(items)
                    print(f"  Page {page + 1}: {len(items)} companies (running total: {len(all_companies)})")
                    
                    if len(items) < 100:  # Last page
                        print(f"  Completed: {len(all_companies)} total companies")
                        break
                    
                    page += 1
                    
                elif response.status_code == 429:  # Rate limit hit
                    print(f"  Rate limit reached. Waiting 60 seconds...")
                    time.sleep(60)
                    continue  # Retry same page
                    
                else:
                    print(f"  Error {response.status_code}: {response.text[:100]}")
                    break
                
                time.sleep(0.5)  # Rate limiting between requests
                
            except Exception as e:
                print(f"  Exception: {str(e)}")
                break
    
    print(f"\n✓ Search complete!")
    print(f"  Total requests made: {total_requests}")
    print(f"  Total unique companies: {len(all_companies)}")
    return all_companies

# Test with same January 2024 range, first 3 codes, but no page limit
test_full_search = search_all_companies(
    '2024-01-01', 
    '2024-01-31', 
    TECH_SIC_CODES[:3]
)


Searching SIC code: 62011
  Page 1: 100 companies (running total: 100)
  Page 2: 100 companies (running total: 200)
  Page 3: 8 companies (running total: 208)
  Completed: 208 total companies

Searching SIC code: 62012
  Page 1: 100 companies (running total: 308)
  Page 2: 100 companies (running total: 408)
  Page 3: 100 companies (running total: 508)
  Page 4: 100 companies (running total: 608)
  Page 5: 100 companies (running total: 708)
  Page 6: 100 companies (running total: 808)
  Page 7: 100 companies (running total: 908)
  Page 8: 100 companies (running total: 1008)
  Page 9: 100 companies (running total: 1108)
  Page 10: 100 companies (running total: 1208)
  Page 11: 100 companies (running total: 1308)
  Page 12: 100 companies (running total: 1408)
  Page 13: 100 companies (running total: 1508)
  Page 14: 100 companies (running total: 1608)
  Page 15: 100 companies (running total: 1708)
  Page 16: 100 companies (running total: 1808)
  Page 17: 100 companies (running total: 190

We now execute the full search across our entire timeframe (2022-2025) and all 21 tech SIC codes. This is our complete raw dataset of UK technology companies. Based on our test run, we estimate this will collect 20,000-30,000 companies and take approximately 5-10 minutes to complete.

In [8]:
# Now run the full search across all years and all SIC codes
print("Starting full data collection: 2022-2025, all 21 SIC codes")
print("This will take approximately 5-10 minutes...\n")

all_uk_tech_companies = search_all_companies(
    incorporated_from='2022-01-01',
    incorporated_to='2025-12-31',
    sic_codes=TECH_SIC_CODES
)

print(f"\n{'='*60}")
print(f"DATA COLLECTION COMPLETE")
print(f"{'='*60}")
print(f"Total companies collected: {len(all_uk_tech_companies):,}")

Starting full data collection: 2022-2025, all 21 SIC codes
This will take approximately 5-10 minutes...


Searching SIC code: 62011
  Page 1: 100 companies (running total: 100)
  Page 2: 100 companies (running total: 200)
  Page 3: 100 companies (running total: 300)
  Page 4: 100 companies (running total: 400)
  Page 5: 100 companies (running total: 500)
  Page 6: 100 companies (running total: 600)
  Page 7: 100 companies (running total: 700)
  Page 8: 100 companies (running total: 800)
  Page 9: 100 companies (running total: 900)
  Page 10: 100 companies (running total: 1000)
  Page 11: 100 companies (running total: 1100)
  Page 12: 100 companies (running total: 1200)
  Page 13: 100 companies (running total: 1300)
  Page 14: 100 companies (running total: 1400)
  Page 15: 100 companies (running total: 1500)
  Page 16: 100 companies (running total: 1600)
  Page 17: 100 companies (running total: 1700)
  Page 18: 100 companies (running total: 1800)
  Page 19: 100 companies (running total:

We define our data extraction function that pulls the key fields we need from each company record: identification (number, name), status, incorporation date, location, and SIC codes. We also timestamp when we retrieved the data.

In [10]:
def extract_company_details(company_item):
    """
    Extract relevant fields from Companies House search results.
    We clean and structure the data for our database.
    """
    return {
        'company_number': company_item.get('company_number'),
        'company_name': company_item.get('company_name'),
        'company_status': company_item.get('company_status'),
        'date_of_creation': company_item.get('date_of_creation'),
        'company_type': company_item.get('company_type'),
        'address': company_item.get('address', {}).get('postal_code', ''),
        'sic_codes': ', '.join(company_item.get('sic_codes', [])),
        'retrieved_at': datetime.now().isoformat()
    }

- We convert our raw API results into a clean pandas DataFrame with 155,277 companies. 

- We examine the data structure, check for active vs dissolved companies, and see the distribution across our target years (2022-2025). 

- This gives us a sense of data quality before we save it to our database.

In [11]:
# Convert to structured DataFrame
print("Converting to DataFrame...")
companies_df = pd.DataFrame([extract_company_details(c) for c in all_uk_tech_companies])

print(f"DataFrame created: {len(companies_df):,} rows × {len(companies_df.columns)} columns")
print(f"\nDataFrame info:")
print(companies_df.info())
print(f"\nFirst few records:")
print(companies_df.head())
print(f"\nCompany status distribution:")
print(companies_df['company_status'].value_counts())
print(f"\nIncorporation year distribution:")
companies_df['incorporation_year'] = pd.to_datetime(companies_df['date_of_creation']).dt.year
print(companies_df['incorporation_year'].value_counts().sort_index())

Converting to DataFrame...
DataFrame created: 155,277 rows × 8 columns

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155277 entries, 0 to 155276
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   company_number    155277 non-null  object
 1   company_name      155277 non-null  object
 2   company_status    155277 non-null  object
 3   date_of_creation  155277 non-null  object
 4   company_type      155277 non-null  object
 5   address           155277 non-null  object
 6   sic_codes         155277 non-null  object
 7   retrieved_at      155277 non-null  object
dtypes: object(8)
memory usage: 9.5+ MB
None

First few records:
  company_number           company_name company_status date_of_creation  \
0       14451923   UNREAL SOLUTIONS LTD         active       2022-10-31   
1       15719845     HORA STARS LIMITED         active       2024-05-14   
2       14299198           MONICONS LTD   